<a href="https://colab.research.google.com/github/Sainath018/PRODIGY_GA_05/blob/main/style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install streamlit

with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
import numpy as np
import altair as alt

# Load the TensorFlow Hub module for style transfer
hub_handle = 'https://www.kaggle.com/models/google/arbitrary-image-stylization-v1/TensorFlow1/256/2'
hub_module = hub.load(hub_handle)

# Function to crop image to a square
def crop_image(img):
    corr_shape = img.shape
    new_shape = min(corr_shape[0], corr_shape[1])
    offset_y = max(corr_shape[0] - corr_shape[1], 0) // 2
    offset_x = max(corr_shape[1] - corr_shape[0], 0) // 2
    img = tf.image.crop_to_bounding_box(img, offset_y, offset_x, new_shape, new_shape)
    return img

# Function to load and preprocess the uploaded image
def load_image(uploaded_file, image_size=(256, 256), col=st):
    img = Image.open(uploaded_file).convert("RGB")  # Convert to RGB to handle transparency
    img = tf.convert_to_tensor(np.array(img), dtype=tf.float32)
    img = crop_image(img)
    img = tf.image.resize(img, image_size)
    img = tf.reshape(img, [-1, image_size[0], image_size[1], 3]) / 255.0  # Normalize
    col.image(np.array(img[0]), use_column_width=True)  # Display the image in the Streamlit app
    return img

# Function to display stylized images
def show_image(images, title=('',), col=st):
    for i in range(len(images)):
        col.image(np.array(images[i]), caption=title[i],  use_container_width=True)

# Streamlit app configuration
st.set_page_config(layout='wide')
alt.renderers.set_embed_options(scaleFactor=2)

if __name__ == '__main__':
    img_width, img_height = 384, 384

    col1, col2 = st.columns(2)

    # Content Image Upload
    content_file = col1.file_uploader("Choose content image")
    if content_file is not None:
        content_image = load_image(content_file, (img_width, img_height), col=col1)

    # Style Image Upload
    style_file = col2.file_uploader("Choose style image")
    if style_file is not None:
        style_image = load_image(style_file, (img_width, img_height), col=col2)
        style_image = tf.nn.avg_pool(style_image, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding='SAME')

        # Apply style transfer
        outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
        stylized_image = outputs[0]

        # Display the output
        col3, col4, col5 = st.columns(3)
        col4.markdown('# The Output')
        show_image([stylized_image], title=['Stylized Image'], col=col4)
    """)


!ngrok authtoken 2oetw8999zj2duV7zVE0lHKWqU8_3LrEeZov9gCYu6aHNwQLB


















Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [33]:
from pyngrok import ngrok
import threading
import os


ngrok.kill()

# Start Streamlit in a separate thread
def run_streamlit():
    os.system("streamlit run app.py")

thread = threading.Thread(target=run_streamlit)
thread.start()

# Open an ngrok tunnel for port 8501 (default Streamlit port)
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

Streamlit app is live at: NgrokTunnel: "https://ba49-34-57-17-101.ngrok-free.app" -> "http://localhost:8501"
